This notebook goes through how to create a **triangle** and a **quadrilateral** from the vertices/**points**.

# IMPORTS

In [ ]:
import matplotlib.pyplot as plt

import plotly.graph_objects as go

from spomso.cores.helper_functions import generate_grid, smarter_reshape, hard_binarization
from spomso.cores.geom_3d import Triangle3D, Quad
from spomso.cores.combine import CombineGeometry

# PARAMETERS

In [ ]:
# size of the volume
co_size = 2.5, 1.5, 2.5
# resolution of the volume
co_resolution = 200, 100, 200

show = "FIELD" # BINARY, FIELD
show_midplane = True
show_3d = True

# COORDINATE SYSTEM

Point cloud of coordinates is created with the **generate_grid** function, which takes the size of the coordinate system and the number of points in each direction (**co_resolution**).

In [ ]:
coor, co_res_new = generate_grid(co_size, co_resolution)

If there is an even number of points along any direction the function will automatically convert it to an odd number, therefore the origin is always included in the point cloud. The point cloud is a numpy.ndarray with a shape of **(D, N)**, where **D** is the number of dimensions and **N** is the number of data points.

# CREATE SDFs

Defined the three vertices/**points** of the triangle:

In [ ]:
point_A = (-1, 0.2)
point_B = (1, -0.2)
point_C = (0.1, 0.5)

Create the **triangle** from the **three points**:

In [ ]:
trig = Triangle3D(point_A, point_B, point_C)

Add thickness to the **triangle**:

In [ ]:
trig.rounding(0.05)

Define the points of the **quadrilateral** (the four points must be in plane):

In [ ]:
point_Q_A = (-1, 0.2, 0)
point_Q_B = (1, -0.2, 0)
point_Q_C = (0.1, 0.5, 0)
point_Q_D = (-0.5, 0.5, 0)

Create the **quadrilateral** from the **four points**:

In [ ]:
quad = Quad(point_Q_A, point_Q_B, point_Q_C, point_Q_D)

Only the **quadrilateral** can be seen in the 2D cross-section.

Add thickness to the **quadrilateral**:

In [ ]:
quad.rounding(0.05)

Create a **union** of both objects:

In [ ]:
union = CombineGeometry("UNION")
combined = union.combine(quad, trig)

## EVALUATE

Evaluate the SDF of the **combined geometry** to create a signed distance field 3D map. This is done by calling the **create** method and passing in the point cloud of coordinates:

In [ ]:
combined_pattern = combined.create(coor)

# BINARIZATION

Convert the distance field to a binary voxel map, where 1 corresponds to the interior and 0 to the exterior of the geometry.

In [ ]:
if show_midplane:
    field = smarter_reshape(combined_pattern, co_resolution)
    if show=="BINARY":
        pattern_2d = hard_binarization(field, 0)

if show=="BINARY":
    pattern = hard_binarization(combined_pattern, 0)

# PLOT

In [ ]:
print("Drawing results...")
# Mid-plane cross-section plot
if show_midplane and show=="BINARY":
    fig, ax = plt.subplots(1,1, figsize=(8.25, 8.25))
    ax.imshow(pattern_2d[:, :, co_resolution[2]//2].T,
              cmap="binary_r",
              extent=(-co_size[0]/2, co_size[0]/2,
                      -co_size[1]/2, co_size[1]/2),
              origin="lower"
              )
    ax.grid()

    fig.tight_layout()
    plt.show()

if show_midplane and show == "FIELD":
    fig, ax = plt.subplots(1, 1, figsize=(8.25, 8.25))
    ax.imshow(field[:, :, co_resolution[2]//2].T,
              cmap="binary_r",
              extent=(-co_size[0] / 2, co_size[0] / 2,
                      -co_size[1] / 2, co_size[1] / 2),
              origin="lower"
              )
    z_mask = coor[2] == 0
    cs = ax.contour(coor[0, z_mask].reshape(co_res_new[0], co_res_new[1]),
                    coor[1, z_mask].reshape(co_res_new[0], co_res_new[1]),
                    field[:, :, co_resolution[2]//2],
                    cmap="plasma_r")
    ax.clabel(cs, inline=True, fontsize=10)
    ax.grid()

    fig.tight_layout()
    plt.show()

In [ ]:
# Isosurfaces plot
if show_3d and show=="BINARY":
    fig = go.Figure(data=go.Volume(
        x=coor[0],
        y=coor[1],
        z=coor[2],
        value=pattern,
        isomin=0.1,
        isomax=1,
        opacity=0.1,
        surface_count=2,
    ))
    fig.show()

if show_3d and show=="FIELD":
    fig = go.Figure(data=go.Volume(
        x=coor[0],
        y=coor[1],
        z=coor[2],
        value=combined_pattern,
        isomin=-0,
        isomax=0.5,
        opacity=0.1,
        surface_count=5,
    ))
    fig.show()